### Importing Libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re, time,csv
from selenium.common.exceptions import NoSuchElementException

### Defining Functions

In [74]:
def scrape(url):
    output='movie_reviews.csv'
    
    file_handler=open(output,'w',encoding='utf8')
    # creates an object of write to append parsed data from selenium to the csv file
    writer=csv.writer(file_handler,lineterminator='\n')
    # appends first row with title, data,...
    writer.writerow(["reviewer's name", 'review date', 'review text', 'review_polarity'])
    
    driver.get(url) # visit the reviews url for the given product
    
    while True: # keep going until there are no more review pages
                 
        #scroll down
        driver.execute_script('window,scrollTo(0,document.body.scrollHeight)')
        # the reason that we need to scroll down the page even though the script is going to find the content and next
        # button is that, there are cases where button might be disabled, like Next button in this case
        # this can be because we are still at the top of the page and thus the button wont be in focus, which in turn
        # may make the button disabled
        # Hence scrolling down changes focus till the end and hence next button wioll be in focus which in turn makes the button enabledvv
        
        #get all the reviews in the page
        reviews=driver.find_elements_by_class_name("review_table_row")
        
        for review in reviews: # for each review
            
            # Initialziing Attributes
            name, date, text, polarity,='NA','NA','NA','NA'
            
            # Name of Author
            try: # try to find the name
                nameBox = review.find_element_by_css_selector('[data-qa="review-critic-link"]')
            except:
                nameBox = None 
                # When inspecting element, we cant consider it as it doestn have the id to uniquely identify it
               
            #box found, extract text
            if nameBox: name=nameBox.text  #retrieves text from titleBox as we dont need other contents 
            
            # Date review was posted
            try: # try to find the date box
                dateBox=review.find_element_by_css_selector('[data-qa="review-date"]')
            except:
                dateBox=None
                
            #box found, extract text
            if dateBox: date=dateBox.text
            
            
            # Review Comment
            try: # try to find the review box
                reviewBox=review.find_element_by_css_selector('[data-qa="review-text"]')
            except NoSuchElementException:
                reviewBox=None
           
            #box found, extract text
            if reviewBox: text = reviewBox.text
                
        
            try: # try to find the rating box
                #ratingBox=review.find_element_by_xpath("div[class^='review_icon icon small']")
                ratingBox = review.find_element_by_css_selector("div[class^='review_icon icon small']")
            except NoSuchElementException:
                ratingBox=None
                
            #box found
            if ratingBox:
                ratingInfo=ratingBox.get_attribute('class') # get the text of class attribute
                #print('rating' + ratingInfo)
                rating = ratingInfo.split(' ')
                #rating=re.search(r'\b[sS]\f+|r+',ratingInfo)  # look for the star rating from the class text
                # re - regular expression
                # group(1) denotes the first parenthesized element. Here the ele we need is what will be coming in place of (\d)
            
                #polarity=rating.group(1) # extract the star rating
                polarity = rating[-1]
            
            #write a new row
            writer.writerow([ name, date, text, polarity])
            
        
        #wait until the next Button loads
            
        nextButton=WebDriverWait(driver,5).until(EC.presence_of_element_located((By.CSS_SELECTOR,'[data-qa="next-btn"]')))
        
        
        if 'hide' in nextButton.get_attribute('class'): # final page reached, 'next' button is disabled on this page
            break
            
        #click on the next Button
        nextButton.click()
        
        #wait for a few seconds
        time.sleep(3)
    

    file_handler.close()

In [52]:
driver = webdriver.Chrome('./chromedriver')

<ipython-input-52-c2e9cad52997>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


In [53]:
driver.maximize_window()

In [55]:
url='https://www.rottentomatoes.com/m/exodus_gods_and_kings/reviews'



In [75]:
scrape(url)

<ipython-input-74-99016b102cb3>:23: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  reviews=driver.find_elements_by_class_name("review_table_row")
